In [11]:
"""
ResumeMate
AI-assisted resume analysis system for resume–job matching,
ATS-inspired optimization, and explainable feedback.

Robust Kaggle/Gemini version
Modeled after CreatorCopilot architecture
"""

# ============================================================
# ResumeMate: Robust Version
# Handles:
# - Missing API keys
# - Kaggle Secrets OR environment variables
# - Model availability fallback
# - Rate limits (429) with retries
# - Gemini extra-text JSON issues (FIXED)
# ============================================================

import os
import time
import random
import json
import google.generativeai as genai

# ------------------ KAGGLE DETECTION ------------------------

try:
    from kaggle_secrets import UserSecretsClient
    KAGGLE_AVAILABLE = True
except ImportError:
    KAGGLE_AVAILABLE = False

# ------------------ API KEY SETUP (SAFE) --------------------

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY and KAGGLE_AVAILABLE:
    try:
        user_secrets = UserSecretsClient()
        GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
        print("API key loaded from Kaggle Secrets.")
    except Exception:
        GEMINI_API_KEY = None

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
else:
    print("WARNING: No API key found. ResumeMate running in MOCK MODE.")

# ------------------ MODEL SELECTION -------------------------

MODEL_ID = "models/gemini-1.5-flash"

if GEMINI_API_KEY:
    try:
        available_models = [m.name for m in genai.list_models()]
        if "models/gemini-2.5-flash" in available_models:
            MODEL_ID = "models/gemini-2.5-flash"
        print(f"Using model: {MODEL_ID}")
    except Exception:
        print("Model listing failed. Falling back to default model.")

# ------------------- LLM CALL HELPER ------------------------

def call_llm(system_prompt: str, user_prompt: str, max_retries: int = 5) -> str:
    """
    Calls Gemini with retry + exponential backoff.
    Returns mock output if API key is missing.
    """

    if not GEMINI_API_KEY:
        return json.dumps({
            "resume_summary": "Mock mode: resume summary unavailable.",
            "job_summary": "Mock mode: job summary unavailable.",
            "extracted_resume_skills": [],
            "required_skills": [],
            "preferred_skills": [],
            "match_score": 0,
            "match_explanation": "API key not configured.",
            "missing_critical_skills": [],
            "missing_secondary_skills": [],
            "rewritten_bullets": [],
            "ats_keyword_suggestions": [],
            "career_recommendations": []
        })

    model = genai.GenerativeModel(
        model_name=MODEL_ID,
        system_instruction=system_prompt
    )

    for attempt in range(max_retries):
        try:
            response = model.generate_content(user_prompt)
            return (response.text or "").strip()

        except Exception as e:
            wait_time = (2 ** attempt) + random.uniform(0, 1)
            print(f"LLM error: {e}. Retrying in {wait_time:.2f}s...")
            time.sleep(wait_time)

    raise RuntimeError("Gemini API failed after multiple retries.")

# ------------------ HARDENED JSON EXTRACTOR (FIX) -----------

def extract_json(text: str) -> dict:
    """
    Safely extracts the FIRST complete JSON object from Gemini output.
    Handles extra text before or after JSON (Gemini 2.5 behavior).
    """

    start = text.find("{")
    if start == -1:
        raise ValueError("No JSON object start found in LLM output.")

    brace_count = 0
    for i in range(start, len(text)):
        if text[i] == "{":
            brace_count += 1
        elif text[i] == "}":
            brace_count -= 1
            if brace_count == 0:
                json_str = text[start:i + 1]
                return json.loads(json_str)

    raise ValueError("Incomplete JSON object returned by LLM.")

# ------------------ SYSTEM PROMPT ---------------------------

SYSTEM_PROMPT = """
You are ResumeMate, an AI-assisted resume analysis system.

STRICT RULES:
- Do NOT invent experience, metrics, or skills.
- Do NOT exaggerate achievements.
- Use ONLY the given resume and job description.
- Be explainable and recruiter-oriented.
- Simulate ATS behavior using heuristics (not a real ATS).
- Avoid keyword stuffing.
- Output VALID JSON ONLY.

TASKS:
1. Summarize the resume
2. Summarize the job description
3. Extract resume skills
4. Extract required and preferred job skills
5. Compute an explainable match score (0–100)
6. Identify missing critical and secondary skills
7. Rewrite weak resume bullets professionally (no fake metrics)
8. Suggest ATS keyword improvements
9. Provide realistic career recommendations
"""

# ------------------ CORE ANALYZER ---------------------------

def resumemate_analyze(resume_text: str, job_description: str) -> dict:
    user_prompt = f"""
RESUME:
{resume_text}

JOB DESCRIPTION:
{job_description}

Return JSON in EXACTLY this structure:
{{
  "resume_summary": "",
  "job_summary": "",
  "extracted_resume_skills": [],
  "required_skills": [],
  "preferred_skills": [],
  "match_score": 0,
  "match_explanation": "",
  "missing_critical_skills": [],
  "missing_secondary_skills": [],
  "rewritten_bullets": [
    {{
      "before": "",
      "after": ""
    }}
  ],
  "ats_keyword_suggestions": [],
  "career_recommendations": []
}}
"""

    raw_output = call_llm(SYSTEM_PROMPT, user_prompt)
    return extract_json(raw_output)

# ------------------ SAMPLE INPUTS ---------------------------

resume_text = """
Skills:
Python, SQL, Git

Experience:
Worked on Python scripts for automation.
Built small data processing tools.

Education:
BSc Computer Science
"""

job_description = """
Backend Developer role.

Required skills:
Python, SQL, REST APIs

Preferred skills:
Docker, Cloud platforms

Responsibilities:
Build scalable backend services and APIs.
"""

# ------------------ RUN RESUMEMATE --------------------------

result = resumemate_analyze(resume_text, job_description)

print("\n========== ResumeMate Analysis Report ==========\n")
print(json.dumps(result, indent=2))


API key loaded from Kaggle Secrets.
Using model: models/gemini-2.5-flash

========== ResumeMate Analysis Report ==========

{
  "resume_summary": "A Computer Science graduate with foundational skills in Python, SQL, and Git. Experience includes developing Python scripts for automation and building small data processing tools.",
  "job_summary": "A Backend Developer position responsible for building scalable backend services and APIs. Requires proficiency in Python, SQL, and REST APIs, with a preference for candidates experienced in Docker and Cloud platforms.",
  "extracted_resume_skills": [
    "Python",
    "SQL",
    "Git"
  ],
  "required_skills": [
    "Python",
    "SQL",
    "REST APIs"
  ],
  "preferred_skills": [
    "Docker",
    "Cloud platforms"
  ],
  "match_score": 50,
  "match_explanation": "The candidate demonstrates a foundational match with core required skills like Python and SQL. However, there is a significant gap in explicit experience with REST APIs, which is a c